In [1]:
### constantes do problema
## 2 => ( 0 - pulmao normal | 1 - pulmao com pneumonia)
numero_classes = 2

## dimensao da imagem
# tamanho x tamanho
tamanho = 100
profundidade = 3

# tamanho do batch
batch = 1500

# numero de epocas
epoch = 100

# quantidade de treino do dataset
treino  = 0.75

## caminhos das pastas das imagens 
caminho_test_normal     = "/home/pedro/Imagens/processado_chest-xray-pneumonia/test/NORMAL/*.png"
caminho_test_pneumonia  = "/home/pedro/Imagens/processado_chest-xray-pneumonia/test/PNEUMONIA/*.png"

caminho_train_normal    = "/home/pedro/Imagens/processado_chest-xray-pneumonia/train/NORMAL/*.png" 
caminho_train_pneumonia = "/home/pedro/Imagens/processado_chest-xray-pneumonia/train/PNEUMONIA/*.png"

caminho_val_normal      = "/home/pedro/Imagens/processado_chest-xray-pneumonia/val/NORMAL/*.png"
caminho_val_pneumonia   = "/home/pedro/Imagens/processado_chest-xray-pneumonia/val/PNEUMONIA/*.png"

In [2]:
import numpy as np
import keras
import cv2 
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint
from keras.regularizers import l2

import matplotlib.pyplot as plt

from stridednet import StridedNet

Using TensorFlow backend.


In [3]:
### funcao para carregar a imagem do caminho

def ler_imagem(caminho):
    img = cv2.imread(caminho)
    imres = image.img_to_array(img)
    imres = np.expand_dims(imres, axis = 0)    
    return imres

In [4]:
### carrega o dataset para a memoria

import glob

arquivo_test_normal    = glob.glob(caminho_test_normal)
arquivo_test_pneumonia = glob.glob(caminho_test_pneumonia)

arquivo_train_normal    = glob.glob(caminho_train_normal)
arquivo_train_pneumonia = glob.glob(caminho_train_pneumonia)

arquivo_val_normal    = glob.glob(caminho_val_normal)
arquivo_val_pneumonia = glob.glob(caminho_val_pneumonia)


## busca cada imagem e salva em arrays

# dados de teste normais
test_normal = []
for caminho_imagem in arquivo_test_normal:
    test_normal.append(ler_imagem(caminho_imagem))
    
# dados de teste com pneumonia
test_pneumonia =  []
for caminho_imagem in arquivo_test_pneumonia:
    test_pneumonia.append(ler_imagem(caminho_imagem))
    
# dados de treino normais
train_normal =  []
for caminho_imagem in arquivo_train_normal:
    train_normal.append(ler_imagem(caminho_imagem))
    
# dados de teste pneumonia
train_pneumonia =  []
for caminho_imagem in arquivo_train_pneumonia:
    train_pneumonia.append(ler_imagem(caminho_imagem))

# dados de validacao normais
val_normal =  []
for caminho_imagem in arquivo_val_normal:
    val_normal.append(ler_imagem(caminho_imagem))
    
# dados de validacao pneumonia
val_pneumonia =  []
for caminho_imagem in arquivo_val_pneumonia:
    val_pneumonia.append(ler_imagem(caminho_imagem))
    

In [5]:
### cria o Y
## 0 -> pulmao normal    1 -> pulmao com pneumonia
y = np.concatenate( 
    (
        np.array( [0] * len(arquivo_test_normal)     ) , 
        np.array( [1] * len(arquivo_test_pneumonia)  ) ,
        np.array( [0] * len(arquivo_train_normal)    ) , 
        np.array( [1] * len(arquivo_train_pneumonia) ) ,
        np.array( [0] * len(arquivo_val_normal)      ) , 
        np.array( [1] * len(arquivo_val_pneumonia)   ) 
    ),
    axis = 0 )

In [6]:
### concatena TODAS as imagens

x = np.concatenate( 
    (
        test_normal, 
        test_pneumonia,
        train_normal, 
        train_pneumonia,
        val_normal, 
        val_pneumonia 
    ),
    axis = 0 )
x = np.array(x, dtype="float") / 255.0
    

# libera memoria
del train_normal    
del train_pneumonia 
del test_normal     
del test_pneumonia  
del val_normal      
del val_pneumonia   

In [7]:
### misturar os dados X e Y

from sklearn.utils import shuffle
x , y  = shuffle(x , y)
  
# rearranja os dados
x = x.reshape(len(x),profundidade,tamanho,tamanho)

print(x.shape)
print(y.shape)

(5856, 3, 100, 100)
(5856,)


In [8]:
### categoriza o Y

y = keras.utils.to_categorical(y , numero_classes)

In [9]:
# divide os dados em teste e treino
from math import floor

x_treino = x[:int(floor(x.shape[0]*treino))]
x_teste  = x[int(floor(x.shape[0]*treino)):]

y_treino = y[:int(floor(y.shape[0]*treino))]
y_teste  = y[int(floor(y.shape[0]*treino)):]


In [10]:
## cria o modelo

modelo = StridedNet(
    width = tamanho, 
    height = tamanho, 
    depth = profundidade , 
    classes = numero_classes, 
    reg = l2(0.0005)
)


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [11]:
## compilacao da cnn
modelo.compile(
    optimizer          = keras.optimizers.Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = None, decay = 0.0, amsgrad = False), 
    loss               = "categorical_hinge", 
    metrics            = ['accuracy'], 
    loss_weights       = None, 
    sample_weight_mode = None, 
    weighted_metrics   = None, 
    target_tensors     = None
)

In [12]:
## cria callback para salvar o modelo a cada epoca finalizada
callback_save = ModelCheckpoint('/home/pedro/Documentos/modelo/xray.h5')

In [ ]:
# treinamento da cnn
history = modelo.fit(
    x                = x_treino, 
    y                = y_treino, 
    batch_size       = batch , 
    epochs           = epoch, 
    verbose          = 1, 
    callbacks        = [callback_save], 
    validation_split = 0.25, 
    validation_data  = None, 
    shuffle          = True, 
    class_weight     = None, 
    sample_weight    = None, 
    initial_epoch    = 0, 
    steps_per_epoch  = None, 
    validation_steps = None
)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 3294 samples, validate on 1098 samples
Epoch 1/100
3294/3294 [==============================] - 145s 44ms/step - loss: 1.2162 - acc: 0.6290 - val_loss: 1.2420 - val_acc: 0.6193
Epoch 2/100
3294/3294 [==============================] - 135s 41ms/step - loss: 0.8518 - acc: 0.8078 - val_loss: 1.8138 - val_acc: 0.3242
Epoch 3/100
3294/3294 [==============================] - 136s 41ms/step - loss: 0.7482 - acc: 0.8625 - val_loss: 1.7823 - val_acc: 0.3388
Epoch 4/100
3294/3294 [==============================] - 134s 41ms/step - loss: 0.7071 - acc: 0.8810 - val_loss: 0.9622 - val_acc: 0.7505
Epoch 5/100
3294/3294 [==============================] - 135s 41ms/step - loss: 0.6507 - acc: 0.9077 - val_loss: 1.9225 - val_acc: 0.2696
Epoch 6/100
3294/3294 [==============================] - 134s 41ms/step - loss: 0.6479 - acc: 0.9098 - val_loss: 1.4504 - val_acc: 0.500

Epoch 56/100
3294/3294 [==============================] - 80s 24ms/step - loss: 0.4246 - acc: 0.9815 - val_loss: 0.7092 - val_acc: 0.8370
Epoch 57/100
3294/3294 [==============================] - 79s 24ms/step - loss: 0.4220 - acc: 0.9812 - val_loss: 0.8704 - val_acc: 0.7559
Epoch 58/100
3294/3294 [==============================] - 79s 24ms/step - loss: 0.4230 - acc: 0.9797 - val_loss: 0.8729 - val_acc: 0.7541
Epoch 59/100
3294/3294 [==============================] - 79s 24ms/step - loss: 0.4183 - acc: 0.9815 - val_loss: 0.8137 - val_acc: 0.7814
Epoch 60/100
3294/3294 [==============================] - 89s 27ms/step - loss: 0.4168 - acc: 0.9815 - val_loss: 0.7094 - val_acc: 0.8342
Epoch 61/100
3294/3294 [==============================] - 96s 29ms/step - loss: 0.4171 - acc: 0.9809 - val_loss: 0.6144 - val_acc: 0.8798
Epoch 62/100
3294/3294 [==============================] - 79s 24ms/step - loss: 0.4117 - acc: 0.9818 - val_loss: 0.5070 - val_acc: 0.9353
Epoch 63/100
3294/3294 [==========

In [ ]:
## mostra resultado
score = modelo.evaluate(x_teste, y_teste, verbose = 1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
## plota resultado
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()